In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut lengden på det nåværende vegnettet for riks- og europaveg i Norge.

In [ ]:
query = f"""
SELECT 
  SUM(ST_LENGTH(geometri))/1000 vegnettKm
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Nåværende vegnett
  (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter ut det nåværende vegnettet for riks- og europaveg i Trøndelag fylke.

In [ ]:
query = f"""
SELECT 
  vegsystemreferanse.kortform, geometri
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan vegeier og vegens navn kan utledes fra de andre feltene.

In [ ]:
query = f"""
SELECT
  FORMAT("%s%s%d", 
    vegsystemreferanse.vegsystem.vegkategori, 
    vegsystemreferanse.vegsystem.fase, 
    vegsystemreferanse.vegsystem.nummer) vegnavn,
  IF(
    EXISTS(SELECT * FROM UNNEST(kontraktsomraader) WHERE REGEXP_CONTAINS(navn, "NV_")),
    "Stat, Nye Veier", 
    "Stat, Statens vegvesen") vegeier,
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()
